In [2]:
import numpy as np


In [8]:
# Mini quiz
# 1 camera projection
R = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
cameraMat = np.array([[725, 0, 631], [0, 726, 360], [0, 0, 1]])
cameraCoor = np.array([1,1,4, 1]).T

Rmarked = np.concatenate((R, np.zeros((3,1))), axis=1)

point = cameraMat@Rmarked@cameraCoor
point /= point[2]
point

# 7
# Calculate the dissimilarity between two images, using sum of squared differences
A = np.array([[10, 15, 20], [20, 20, 25], [10, 15, 20]])
B = np.array([[15, 15, 15], [20, 20, 20], [30, 30, 30]])
sum = np.sum(np.square(A-B))
sum



800